<a href="https://colab.research.google.com/github/iamRahulB/Advanced-Hierarchical-Semantic-Search-Retrieval-System/blob/main/TEXT/Word2Vec_from_scratch_CBOW_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -q torch==2.2.0 torchtext==0.17.0 --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.7/811.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
!pip install -q -U portalocker==2.8.2

In [3]:
import requests
import zipfile
import re
import torch
from torch import nn
import torchtext
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from sklearn.metrics import accuracy_score
from torchtext import datasets
from torch.utils.data import DataLoader, Dataset
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import os
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pad_packed_sequence


In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
response=requests.get('https://www.kaggle.com/api/v1/datasets/download/khulasasndh/game-of-thrones-books')

with open('data.zip','wb') as f:
    f.write(response.content)
    print('downloaded')

with zipfile.ZipFile('data.zip','r') as f_z:
    f_z.extractall('data')
    print('extraction done')

downloaded
extraction done


In [6]:
lemma=WordNetLemmatizer()
stop_words=stopwords.words("english")

In [7]:
def preprocess(text):
    text = re.sub(r'<[^>]+>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-z\s.]', '', text, flags=re.IGNORECASE)  # Keep dots
    text = [lemma.lemmatize(t) for t in word_tokenize(text) if t.lower() not in stop_words]
    return " ".join(text)

In [8]:
all_sentences = []

for files in os.listdir('data'):
    file_name = os.path.join('data', files)
    with open(file_name, 'r', encoding='latin-1') as f:
        content = f.read()
        cleaned = preprocess(content)
        cleaned_sentences = sent_tokenize(cleaned)
        all_sentences.extend(cleaned_sentences)

In [9]:

tokenized_sentences = [word_tokenize(sentence) for sentence in all_sentences]

In [10]:
len(tokenized_sentences)

143966

In [11]:
tokenized_sentences[0]

['Storm',
 'Swords',
 'Book',
 'Three',
 'Song',
 'Ice',
 'Fire',
 'George',
 'R.R',
 '.']

In [14]:
from itertools import chain

def generate_context_target_pairs(tokenized_sentences, window_size=10):
    context_target_pairs = []
    for sentence in tokenized_sentences:
        for i in range(len(sentence)):
            if sentence[i] == '.':
                continue  # Skip if the token is a period
            # Define the context window
            start = max(0, i - window_size)
            end = min(len(sentence), i + window_size + 1)
            # Extract the context words
            context = [word for word in (sentence[start:i] + sentence[i+1:end]) if word != '.']
            # The target word
            target = sentence[i]
            context_target_pairs.append((context, target))
    return context_target_pairs

context_target_pairs = generate_context_target_pairs(tokenized_sentences)





In [15]:
for context, target in context_target_pairs[:10]:
    print(f"Context: {context}, Target: {target}")

Context: ['Swords', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'George', 'R.R'], Target: Storm
Context: ['Storm', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'George', 'R.R'], Target: Swords
Context: ['Storm', 'Swords', 'Three', 'Song', 'Ice', 'Fire', 'George', 'R.R'], Target: Book
Context: ['Storm', 'Swords', 'Book', 'Song', 'Ice', 'Fire', 'George', 'R.R'], Target: Three
Context: ['Storm', 'Swords', 'Book', 'Three', 'Ice', 'Fire', 'George', 'R.R'], Target: Song
Context: ['Storm', 'Swords', 'Book', 'Three', 'Song', 'Fire', 'George', 'R.R'], Target: Ice
Context: ['Storm', 'Swords', 'Book', 'Three', 'Song', 'Ice', 'George', 'R.R'], Target: Fire
Context: ['Storm', 'Swords', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'R.R'], Target: George
Context: ['Storm', 'Swords', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'George'], Target: R.R
Context: ['NOTE', 'CHRONOLOGY', 'Song', 'Ice', 'Fire', 'told', 'eye', 'character', 'sometimes', 'hundred'], Target: Martin


In [16]:
import pandas as pd

In [17]:
print(context_target_pairs[:10])

[(['Swords', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'George', 'R.R'], 'Storm'), (['Storm', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'George', 'R.R'], 'Swords'), (['Storm', 'Swords', 'Three', 'Song', 'Ice', 'Fire', 'George', 'R.R'], 'Book'), (['Storm', 'Swords', 'Book', 'Song', 'Ice', 'Fire', 'George', 'R.R'], 'Three'), (['Storm', 'Swords', 'Book', 'Three', 'Ice', 'Fire', 'George', 'R.R'], 'Song'), (['Storm', 'Swords', 'Book', 'Three', 'Song', 'Fire', 'George', 'R.R'], 'Ice'), (['Storm', 'Swords', 'Book', 'Three', 'Song', 'Ice', 'George', 'R.R'], 'Fire'), (['Storm', 'Swords', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'R.R'], 'George'), (['Storm', 'Swords', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'George'], 'R.R'), (['NOTE', 'CHRONOLOGY', 'Song', 'Ice', 'Fire', 'told', 'eye', 'character', 'sometimes', 'hundred'], 'Martin')]


In [18]:
df=pd.DataFrame(context_target_pairs,columns=['text','target'])

In [17]:
df

,text,target
0,"[Swords, Book, Three, Song, Ice, Fire, George,...",Storm
1,"[Storm, Book, Three, Song, Ice, Fire, George, ...",Swords
2,"[Storm, Swords, Three, Song, Ice, Fire, George...",Book
3,"[Storm, Swords, Book, Song, Ice, Fire, George,...",Three
4,"[Storm, Swords, Book, Three, Ice, Fire, George...",Song
...,...,...
1069584,"[QHORIN, called, HALFHAND, senior, ranger, DAL...",EBBEN
1069585,"[called, HALFHAND, senior, ranger, DALBRIDGE, ...",STONESNAKE
1069586,"[HALFHAND, senior, ranger, DALBRIDGE, elderly,...",ranger
1069587,"[senior, ranger, DALBRIDGE, elderly, squire, s...",.


In [19]:
df['text'].tolist()[0:10]

[['Swords', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'George', 'R.R'],
 ['Storm', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'George', 'R.R'],
 ['Storm', 'Swords', 'Three', 'Song', 'Ice', 'Fire', 'George', 'R.R'],
 ['Storm', 'Swords', 'Book', 'Song', 'Ice', 'Fire', 'George', 'R.R'],
 ['Storm', 'Swords', 'Book', 'Three', 'Ice', 'Fire', 'George', 'R.R'],
 ['Storm', 'Swords', 'Book', 'Three', 'Song', 'Fire', 'George', 'R.R'],
 ['Storm', 'Swords', 'Book', 'Three', 'Song', 'Ice', 'George', 'R.R'],
 ['Storm', 'Swords', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'R.R'],
 ['Storm', 'Swords', 'Book', 'Three', 'Song', 'Ice', 'Fire', 'George'],
 ['NOTE',
  'CHRONOLOGY',
  'Song',
  'Ice',
  'Fire',
  'told',
  'eye',
  'character',
  'sometimes',
  'hundred']]

In [20]:
def joins(text):
    return ' '.join(text)


In [21]:
df['text_join']=df['text'].apply(joins)

In [22]:
df

,text,target,text_join
0,"[Swords, Book, Three, Song, Ice, Fire, George,...",Storm,Swords Book Three Song Ice Fire George R.R
1,"[Storm, Book, Three, Song, Ice, Fire, George, ...",Swords,Storm Book Three Song Ice Fire George R.R
2,"[Storm, Swords, Three, Song, Ice, Fire, George...",Book,Storm Swords Three Song Ice Fire George R.R
3,"[Storm, Swords, Book, Song, Ice, Fire, George,...",Three,Storm Swords Book Song Ice Fire George R.R
4,"[Storm, Swords, Book, Three, Ice, Fire, George...",Song,Storm Swords Book Three Ice Fire George R.R
...,...,...,...
925681,"[Tower, QHORIN, called, HALFHAND, senior, rang...",ranger,Tower QHORIN called HALFHAND senior ranger DAL...
925682,"[QHORIN, called, HALFHAND, senior, ranger, DAL...",EBBEN,QHORIN called HALFHAND senior ranger DALBRIDGE...
925683,"[called, HALFHAND, senior, ranger, DALBRIDGE, ...",STONESNAKE,called HALFHAND senior ranger DALBRIDGE elderl...
925684,"[HALFHAND, senior, ranger, DALBRIDGE, elderly,...",ranger,HALFHAND senior ranger DALBRIDGE elderly squir...


In [23]:
tokenizer=get_tokenizer('basic_english')

In [24]:
def build_vocab(texts):
    for text in texts:
        yield tokenizer(text)


In [25]:
vocab=build_vocab_from_iterator(build_vocab(df['text_join'].tolist()),specials=['<unk>'])
vocab.set_default_index(0)

In [30]:
vocab['range'],len(vocab)

(4602, 23584)

In [31]:
text_pipeline=lambda x: vocab(tokenizer(x))
label_pipeline=lambda x: int(x)


In [32]:
text_pipeline("hey there")

[13681, 922]

In [33]:
def make_num(text):
    return text_pipeline(text)

In [34]:
df['text_num']=df['text_join'].apply(make_num)
df['target']=df['target'].apply(make_num)

In [35]:
df

,text,target,text_join,text_num
0,"[Swords, Book, Three, Song, Ice, Fire, George,...",[443],Swords Book Three Song Ice Fire George R.R,"[2976, 599, 60, 373, 391, 104, 3989, 2089, 2, ..."
1,"[Storm, Book, Three, Song, Ice, Fire, George, ...",[2976],Storm Book Three Song Ice Fire George R.R,"[443, 599, 60, 373, 391, 104, 3989, 2089, 2, 2..."
2,"[Storm, Swords, Three, Song, Ice, Fire, George...",[599],Storm Swords Three Song Ice Fire George R.R,"[443, 2976, 60, 373, 391, 104, 3989, 2089, 2, ..."
3,"[Storm, Swords, Book, Song, Ice, Fire, George,...",[60],Storm Swords Book Song Ice Fire George R.R,"[443, 2976, 599, 373, 391, 104, 3989, 2089, 2,..."
4,"[Storm, Swords, Book, Three, Ice, Fire, George...",[373],Storm Swords Book Three Ice Fire George R.R,"[443, 2976, 599, 60, 391, 104, 3989, 2089, 2, ..."
...,...,...,...,...
925681,"[Tower, QHORIN, called, HALFHAND, senior, rang...",[605],Tower QHORIN called HALFHAND senior ranger DAL...,"[130, 1484, 50, 1644, 3879, 605, 5174, 7071, 2..."
925682,"[QHORIN, called, HALFHAND, senior, ranger, DAL...",[5175],QHORIN called HALFHAND senior ranger DALBRIDGE...,"[1484, 50, 1644, 3879, 605, 5174, 7071, 234, 3..."
925683,"[called, HALFHAND, senior, ranger, DALBRIDGE, ...",[3227],called HALFHAND senior ranger DALBRIDGE elderl...,"[50, 1644, 3879, 605, 5174, 7071, 234, 3879, 6..."
925684,"[HALFHAND, senior, ranger, DALBRIDGE, elderly,...",[605],HALFHAND senior ranger DALBRIDGE elderly squir...,"[1644, 3879, 605, 5174, 7071, 234, 3879, 605, ..."


In [36]:
df.drop(columns=['text','text_join'],inplace=True,axis=1)

In [37]:
df

,target,text_num
0,[443],"[2976, 599, 60, 373, 391, 104, 3989, 2089, 2, ..."
1,[2976],"[443, 599, 60, 373, 391, 104, 3989, 2089, 2, 2..."
2,[599],"[443, 2976, 60, 373, 391, 104, 3989, 2089, 2, ..."
3,[60],"[443, 2976, 599, 373, 391, 104, 3989, 2089, 2,..."
4,[373],"[443, 2976, 599, 60, 391, 104, 3989, 2089, 2, ..."
...,...,...
925681,[605],"[130, 1484, 50, 1644, 3879, 605, 5174, 7071, 2..."
925682,[5175],"[1484, 50, 1644, 3879, 605, 5174, 7071, 234, 3..."
925683,[3227],"[50, 1644, 3879, 605, 5174, 7071, 234, 3879, 6..."
925684,[605],"[1644, 3879, 605, 5174, 7071, 234, 3879, 605, ..."


In [71]:
df['target']=df['target'].apply(lambda x: x[0])

In [72]:
df

,target,text_num
0,443,"[2976, 599, 60, 373, 391, 104, 3989, 2089, 2, ..."
1,2976,"[443, 599, 60, 373, 391, 104, 3989, 2089, 2, 2..."
2,599,"[443, 2976, 60, 373, 391, 104, 3989, 2089, 2, ..."
3,60,"[443, 2976, 599, 373, 391, 104, 3989, 2089, 2,..."
4,373,"[443, 2976, 599, 60, 391, 104, 3989, 2089, 2, ..."
...,...,...
925681,605,"[130, 1484, 50, 1644, 3879, 605, 5174, 7071, 2..."
925682,5175,"[1484, 50, 1644, 3879, 605, 5174, 7071, 234, 3..."
925683,3227,"[50, 1644, 3879, 605, 5174, 7071, 234, 3879, 6..."
925684,605,"[1644, 3879, 605, 5174, 7071, 234, 3879, 605, ..."


In [73]:
df['text_num'][4]

[443, 2976, 599, 60, 391, 104, 3989, 2089, 2, 2089]

In [34]:
# make custom class for dataset to make it iterable
#  note we flattened the above target from list as we dont want to store single num in list but we keep text_num in list as we want it

In [74]:
class CustomDataset(Dataset):
    def __init__(self,text,label):
        super().__init__()
        self.text=text
        self.label=label

    def __len__(self):
        return len(self.text)

    def __getitem__(self,idx):

        return self.text[idx],self.label[idx]

In [75]:
# we dont need to split the dataset when dealing with to mke unsupervided model which learns from our data

train_iter=CustomDataset(df['text_num'].tolist(),df['target'].tolist())

In [92]:
def collate_batch(batch):
    all_contexts = []
    all_targets = []
    for context, target in batch:
        all_contexts.append(torch.tensor(context, dtype=torch.long))
        all_targets.append(label_pipeline(target))

    all_contexts = pad_sequence(all_contexts, batch_first=True, padding_value=0)
    all_targets = torch.tensor(all_targets, dtype=torch.long)
    return all_contexts, all_targets

In [93]:
#  now we will make a dataloader

train_loader=DataLoader(train_iter,batch_size=32,shuffle=True,collate_fn=collate_batch,)

In [94]:
train_loader

In [95]:
next(iter(train_loader))

(tensor([[ 2958,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0],
         [   40, 14523,    22,  2551,    90,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0],
         [ 2468,    76,  4449,   921,    37,   249,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0],
         [   64,  2246,   141,    46,   249,  1708, 11530,  9296,  9317,     0,
              0,     0,     0,     0,     0,     0,     0],
         [  216,   126,  8052,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0],
         [ 2657,  3434,    12,  1911, 17499,   487,   367,   720,    54,   130,
            705,   419,     0,     0,     0,     0,     0],
         [ 4932,    37,    82,   990,    93,  1285,    32,    64,  3678,     0,
              0,     0,     0,     0,     0,     0,     0],
         [    1,    

In [45]:
# import torch

# # Example trained model
# class Word2VecModel(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
#         super(Word2VecModel, self).__init__()
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         self.rnn = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
#         self.linear = nn.Linear(hidden_size, vocab_size)

#     def forward(self, x, lengths):
#         embedded = self.embeddings(x)
#         packed_embedded = pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=True)
#         packed_output, (hidden, cell) = self.rnn(packed_embedded)
#         output, _ = pad_packed_sequence(packed_output, batch_first=True)
#         output = self.linear(output)
#         return output


In [59]:
len(vocab)

23584

In [96]:
with torch.inference_mode():
    o=model(next(iter(train_loader))[0].to(device))

In [97]:
o

tensor([[ -1.3339,   1.5373,   2.9632,  ..., -15.0657, -15.7177, -12.4635],
        [ -0.9584,   9.0672,   9.2826,  ..., -10.3440, -13.8112, -13.6959],
        [ -3.5507,   2.8915,   2.0301,  ..., -17.3786, -13.7703, -12.6006],
        ...,
        [ -1.4625,  12.5565,  12.1599,  ...,  -5.2821, -13.2563,  -8.5015],
        [ -2.9486,   3.7474,   1.4898,  ..., -13.9005, -14.6148, -12.4483],
        [ -4.9716,  12.2091,   7.3116,  ...,  -7.5957, -11.5179, -10.5776]],
       device='cuda:0')

In [115]:
class Word2VecModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=100):
        super(Word2VecModel, self).__init__()
        self.embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.linear = nn.Linear(in_features=embedding_dim, out_features=vocab_size)  # Output should match vocab size

    def forward(self, x):
        # Sum the embeddings of context words
        embed_out = self.embed(x).sum(dim=1)
        out = self.linear(embed_out)
        return out

In [116]:
# heree we took out features = vocab_size because linear layer will make probability over entire vocab  also we will not use this layer to generate embedding as we will use embed layer
#  why we use vocab size as out features ? if we extract max num from label tht will be vocab size, if we had 4 classes then label will have only 4 classes . so

#  above model try to learn from text sequences and labels and to get embedding of perticular word we will use only embedding layer nothing else

In [117]:
device=('cuda' if torch.cuda.is_available() else 'cpu')

In [118]:
import torch.optim as optim
from tqdm.auto import tqdm


In [119]:
vocab_size=len(vocab)

In [120]:

model = Word2VecModel(len(vocab),100).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.01)
epochs = 2

for epoch in tqdm(range(epochs), desc="Epochs"):
    train_loss = 0.0
    train_acc = 0.0
    model.train()

    for batch_idx, (X_train, y_train) in tqdm(enumerate(train_loader), desc='Training Mode', leave=False):
        X_train, y_train = X_train.to(device), y_train.to(device)

        # Forward pass
        train_batch_logits = model(X_train)

        # Compute loss
        train_batch_loss = loss_fn(train_batch_logits, y_train)  # Ensure y_train is flat

        # Predictions and accuracy
        train_batch_preds = train_batch_logits.argmax(dim=1)
        train_batch_acc = accuracy_score(y_train.cpu().numpy(), train_batch_preds.cpu().numpy())

        # Backward pass and optimization
        optimizer.zero_grad()
        train_batch_loss.backward()
        optimizer.step()

        train_loss += train_batch_loss.item()
        train_acc += train_batch_acc

    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Epoch: {epoch+1} | Train Loss: {train_loss:.4f} | Train ACC: {train_acc*100:.2f}")

print("Training complete.")

Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

Training Mode: 0it [00:00, ?it/s]

Epoch: 1 | Train Loss: 11.2709 | Train ACC: 1.41


Training Mode: 0it [00:00, ?it/s]

Epoch: 2 | Train Loss: 10.6633 | Train ACC: 2.06
Training complete.


In [123]:
model.embed.weight.data

tensor([[ 0.0267, -0.0047,  0.0026,  ...,  0.0254,  0.0229, -0.0182],
        [-0.0083,  0.0118,  0.4495,  ..., -0.4775,  0.4680, -0.2900],
        [-0.0174,  0.0612,  0.0832,  ..., -0.0395, -0.0496, -0.1259],
        ...,
        [ 0.2773,  0.1158,  1.1285,  ..., -1.0799, -0.3410, -0.5736],
        [ 0.5672,  1.9275,  1.3310,  ...,  0.6428, -0.2049,  0.7953],
        [ 0.6298, -0.8674,  0.8130,  ..., -0.2258, -0.3944,  1.0765]],
       device='cuda:0')

In [126]:
from sklearn.metrics.pairwise import cosine_similarity
import torch

def get_word_embedding(model, word, text_pipeline, device):
    model.eval()
    with torch.no_grad():
        # Tokenize and convert word to tensor
        word_tensor = torch.tensor(text_pipeline(word), dtype=torch.long).to(device)
        # Get embedding and ensure it's 2D
        embedding = model.embed(word_tensor).squeeze(0).cpu().numpy()
    return embedding

def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Example usage:
word1, word2 = "king", "queen"
embedding1 = get_word_embedding(model, word1, text_pipeline, device)
embedding2 = get_word_embedding(model, word2, text_pipeline, device)
similarity_score = calculate_cosine_similarity(embedding1, embedding2)

print(f"Cosine Similarity between '{word1}' and '{word2}': {similarity_score}")


Cosine Similarity between 'king' and 'queen': 0.09012425690889359
